# YOLO Sigma-Delta Neural Network (SDNN) Inference


In [ ]:
import os
from datetime import datetime
import torch
import yaml

from torch.utils.data import DataLoader

from lava.lib.dl import slayer
from lava.lib.dl.slayer import obd

# Load Inference parameters

In [ ]:
args = slayer.utils.dotdict(load='trained/network.pt')
trained_folder = os.path.dirname(args.load)
print(trained_folder)

with open(trained_folder + '/args.txt', 'rt') as f:
    model_args = slayer.utils.dotdict(yaml.safe_load(f))
    for (k, v) in model_args.items():
        if k not in args.keys():
            args[k] = v
            
print('Using GPUs {}'.format(args.gpu))
device = torch.device('cuda:{}'.format(args.gpu[0]))
classes_output = {'BDD100K': 11}
print(args)

# Load Network Parameters

In [ ]:
if len(args.gpu) == 1:
        net = obd.models.tiny_yolov3_str.Network(threshold=model_args.threshold,
                        tau_grad=model_args.tau_grad,
                        scale_grad=model_args.scale_grad,
                        num_classes=classes_output[model_args.dataset],
                        clamp_max=model_args.clamp_max).to(device)
        module = net
else:
        net = torch.nn.DataParallel(obd.models.tiny_yolov3_str.Network(threshold=model_args.threshold,
                                        tau_grad=model_args.tau_grad,
                                        scale_grad=model_args.scale_grad,
                                        num_classes=classes_output[model_args.dataset],
                                        clamp_max=model_args.clamp_max).to(device),
                                device_ids=args.gpu)
        module = net.module

module.init_model((448, 448))
module.load_state_dict(torch.load(args.load))

yolo_target = obd.YOLOtarget(anchors=net.anchors,
                             scales=net.scale,
                             num_classes=net.num_classes,
                             ignore_iou_thres=model_args.tgt_iou_thr)

# Load Dataset and Dataloader

In [ ]:
test_set = obd.dataset.BDD(root=args.path, dataset='track', train=False, randomize_seq=False, seq_len=60)

test_loader = DataLoader(test_set,
                        batch_size=args.b,
                        shuffle=True,
                        collate_fn=yolo_target.collate_fn,
                        num_workers=args.num_workers,
                        pin_memory=True)

# Run Inference

In [ ]:
epoch = 0
stats = slayer.utils.LearningStats(accuracy_str='AP@0.5')
t_st = datetime.now()
ap_stats = obd.bbox.metrics.APstats(iou_threshold=0.5)
all_counts = []
for i, (inputs, targets, bboxes) in enumerate(test_loader):
    net.eval()

    with torch.no_grad():
        inputs = inputs.to(device)
        predictions, counts = net(inputs)
        all_counts.append(counts.cpu().data.numpy())

        T = inputs.shape[-1]
        
        predictions = [obd.bbox.utils.non_maximum_suppression(predictions[..., t]) for t in range(T)]

        for t in range(T):
            ap_stats.update(predictions[t], bboxes[t])

        # stats.testing.loss_sum += loss.item() * inputs.shape[0]
        stats.testing.num_samples += inputs.shape[0]
        stats.testing.correct_samples = ap_stats[:] * stats.testing.num_samples

        processed = i * test_loader.batch_size
        total = len(test_loader.dataset)
        time_elapsed = (datetime.now() - t_st).total_seconds()
        samples_sec = time_elapsed / (i + 1) / test_loader.batch_size
        header_list = [f'Test: [{processed}/{total} '
                        f'({100.0 * processed / total:.0f}%)]']
        header_list += ['Event Rate: ['
                        + ', '.join([f'{c.item():.2f}'
                                        for c in counts[0]]) + ']']
        
        print(f'\r{i}, {samples_sec:3f}samples/sec {stats}', end='')

# Results: Output vs groundtruth

In [ ]:
from IPython.display import Video

obd.bbox.utils.create_video(inputs, bboxes, predictions, 'trained/test', test_set.classes)

Video("./trained/test.mp4", embed=True)
